In [1]:
import numpy as np

import pandas as pd

import random

from numpy.random import RandomState

import os

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

from sklearn import preprocessing

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler

from sklearn import linear_model

from sklearn.linear_model import Lasso, LassoCV

from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor

from sklearn.neural_network import MLPRegressor

from xgboost.sklearn import XGBRegressor

from lightgbm import LGBMRegressor

from scipy.optimize import minimize

from sklearn.metrics import mean_squared_error as mse

from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, KFold

from sklearn import metrics

from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

import time

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_boston

from sklearn.feature_selection import RFE

import warnings

from scipy.io import loadmat

from sklearn.model_selection import LeavePGroupsOut, GridSearchCV, GroupKFold

from eli5.sklearn import PermutationImportance

from hyperopt import STATUS_OK 

from hyperopt import hp 

from hyperopt import tpe 

from hyperopt import Trials 

from hyperopt import fmin 

import os

from pathlib import Path

from math import sqrt
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

In [2]:
np.random.seed(1607)

In [3]:
cv = 10

#Importing Preprocessed data

data= pd.read_csv('Corn_Yield_12States_Final_Data.csv',index_col= 0)
data.head()

,Yield,Year,State,State_Name,County,County_Name,Lat,Long,S_var1_depth1,S_var1_depth2,...,SW15cm_May,SW15cm_Jun,SW15cm_Jul,SW15cm_Aug,SW15cm_Sep,SW15cm_Oct,WTatPlanting,SW45_excess,SW45_deficit,yield_trend
0,201.6,2020,17,ILLINOIS,39,DE WITT,40.173477,-88.900373,1.332400,1.332400,...,0.6490,0.3475,0.3825,0.2790,0.2885,0.5805,586.7115,89.1745,33.9575,207.692984
1,210.7,2020,17,ILLINOIS,107,LOGAN,40.107509,-89.376854,1.333889,1.333889,...,0.6050,0.2360,0.3380,0.3160,0.4390,0.5790,848.0450,70.1850,27.2440,202.875238
2,210.3,2020,17,ILLINOIS,115,MACON,39.845521,-88.960238,1.396250,1.396250,...,0.6875,0.3770,0.3970,0.2970,0.2700,0.5140,495.9950,94.8340,33.0530,208.297619
3,207.1,2020,17,ILLINOIS,123,MARSHALL,41.032454,-89.299846,1.330400,1.340000,...,0.6100,0.3085,0.3630,0.2295,0.5175,0.5920,769.0720,86.8000,28.3820,209.087619
4,195.6,2020,17,ILLINOIS,125,MASON,40.230227,-89.866043,1.429231,1.429231,...,0.4145,0.1620,0.3510,0.2665,0.3725,0.4340,2074.5635,36.5720,24.2305,180.685748


In [4]:
data.columns

Index(['Yield', 'Year', 'State', 'State_Name', 'County', 'County_Name', 'Lat',
       'Long', 'S_var1_depth1', 'S_var1_depth2',
       ...
       'SW15cm_May', 'SW15cm_Jun', 'SW15cm_Jul', 'SW15cm_Aug', 'SW15cm_Sep',
       'SW15cm_Oct', 'WTatPlanting', 'SW45_excess', 'SW45_deficit',
       'yield_trend'],
      dtype='object', length=335)

In [5]:
# Splitting the data set to test and train

test = data[data.Year==2020]

train = data[data.Year!=2020]
x_test_unscaled = test.drop(columns=['Yield','State_Name','County_Name'])

y_test = test.Yield

X_unscaled = train.drop(columns=['Yield','State_Name','County_Name'])

X_unscaled = X_unscaled.reset_index(drop=True)

Y = train.Yield

Y.reset_index(inplace=True, drop=True)

In [7]:
# Scaling the variables

## First spilt data and then scale the train
columns_to_scale = X_unscaled.drop(columns=['Year','State','County']).columns.values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_unscaled[columns_to_scale]) #training
X = pd.DataFrame(X_scaled, columns=columns_to_scale)
X = pd.concat([X_unscaled.Year,X], axis=1)
# scaler.transform for test
#Scaling test set 
x_test_scaled = scaler.transform(x_test_unscaled[columns_to_scale])
x_test = pd.DataFrame(x_test_scaled, columns=columns_to_scale)
x_test = pd.concat([x_test_unscaled.Year,x_test], axis=1)

In [449]:
# feature selection with random forest

rf = RandomForestRegressor(n_estimators=300)

rf.fit(X, Y)


RandomForestRegressor(n_estimators=300)

In [450]:
perm = PermutationImportance(rf, cv=cv, n_iter=10).fit(X, Y)

feature_importances = [(feature, importance) for feature, importance in zip(list(X.columns), list(np.abs(perm.feature_importances_)))]
# Sort data Frame
feature_importances = pd.DataFrame(sorted(feature_importances, key = lambda x: x[1], reverse = True))

selected_features = feature_importances.iloc[0:100,:][0] #trail and error for 80

if np.isin('Year', selected_features)==False:

    selected_features = selected_features.append(pd.Series('Year'))

X = X.loc[:,selected_features]

x_test = x_test.loc[:,selected_features]

selected_features.to_csv('RF_features_2020_APSIM.csv')

In [451]:
feature_importances.to_csv('Feature_Ranked_Corn_2020_APSIM.csv')

In [497]:
kf = KFold(cv)
max_evals = 40

In [ ]:
## ---------------- Permutation feature importance ---------------- ##

def perm_fi(model, cv, n_iter):

    perm = PermutationImportance(model, cv=cv, n_iter=n_iter).fit(X.drop(columns='Year'), Y)

    feature_importances = [(feature, importance) for feature, importance in zip(list(X.columns), list(np.abs(perm.feature_importances_)))]

    feature_importances = pd.DataFrame(sorted(feature_importances, key = lambda x: x[1], reverse = True))

    return feature_importances

In [471]:
def objective_XGB(params):

    XGB_df_B = pd.DataFrame()

    X1_B = XGBRegressor(objective='reg:squarederror', **params)

    for train_index, test_index in kf.split(X):

        XGB_B = X1_B.fit(np.array(X.drop(columns='Year'))[train_index], np.array(Y)[train_index])

        XGB_df_B = pd.concat([XGB_df_B, pd.DataFrame(X1_B.predict(np.array(X.drop(columns='Year'))[test_index]))])

    loss_XGB = mse(data.Yield[(data.Year < 2020)], XGB_df_B)

    return {'loss': loss_XGB, 'params': params, 'status': STATUS_OK}

In [472]:
space_XGB = {'gamma': hp.uniform('gamma', 0, 1),

             'learning_rate': hp.uniform('learning_rate', 0.003, 0.5),

             'n_estimators': hp.choice('n_estimators', [100, 300, 500]),

             'max_depth': hp.choice('max_depth', [int(x) for x in np.arange(3, 12, 2)])}

tpe_algorithm = tpe.suggest

trials_XGB = Trials()

best_XGB = fmin(fn=objective_XGB, space=space_XGB, algo=tpe.suggest,

                max_evals=max_evals, trials=trials_XGB, rstate=RandomState(1607))

XGB_param_B = pd.DataFrame({'gamma': [], 'learning_rate': [], 'n_estimators': [], 'max_depth': []})

for i in range(max_evals):

    XGB_param_B.gamma[i] = trials_XGB.results[i]['params']['gamma']

    XGB_param_B.learning_rate[i] = trials_XGB.results[i]['params']['learning_rate']

    XGB_param_B.n_estimators[i] = trials_XGB.results[i]['params']['n_estimators']

    XGB_param_B.max_depth[i] = trials_XGB.results[i]['params']['max_depth']

XGB_param_B = pd.DataFrame({'gamma': XGB_param_B.gamma,

                            'learning_rate': XGB_param_B.learning_rate,

                            'n_estimators': XGB_param_B.n_estimators,

                            'max_depth': XGB_param_B.max_depth})

100%|████████████████████████████████████████████| 50/50 [18:29:36<00:00, 1331.53s/trial, best loss: 259.5580862022888]


In [476]:
### ---------- XGB ------------ ###

XGB_df2 = pd.DataFrame()

X2 = XGBRegressor(objective='reg:squarederror',

                  gamma=trials_XGB.best_trial['result']['params']['gamma'],

                  learning_rate=trials_XGB.best_trial['result']['params']['learning_rate'],

                  n_estimators=int(trials_XGB.best_trial['result']['params']['n_estimators']),

                  max_depth=int(trials_XGB.best_trial['result']['params']['max_depth']), random_state=1369)

for train_index, test_index in kf.split(X):

    X2.fit(np.array(X.drop(columns='Year'))[train_index], np.array(Y)[train_index])

    XGB_df2 = pd.concat([XGB_df2, pd.DataFrame(X2.predict(np.array(X.drop(columns='Year'))[test_index]))])

XGB_df2 = XGB_df2.reset_index(drop=True)

XGB_mse2 = mse(data.Yield[(data.Year<2020)], XGB_df2)
XGB_rmse2 = np.sqrt(XGB_mse2)

XGB = X2.fit(X.drop(columns='Year'), Y)

XGB_preds_test2 = XGB.predict(x_test.drop(columns='Year'))

pd.DataFrame(XGB_preds_test2).to_csv('XGB_preds_test_2020_APSIM.csv')

XGB_mse_test2 = mse(data.Yield[data.Year==2020], XGB_preds_test2)

XGB_rmse_test2 = np.sqrt(XGB_mse_test2)

XGB_preds_train = XGB.predict(X.drop(columns='Year'))

pd.DataFrame(XGB_preds_train).to_csv('XGB_preds_train_2020_APSIM.csv')

XGB_rmse_train = np.sqrt(mse(data.Yield[data.Year<2020], XGB_preds_train))

In [62]:
perm_xgb = PermutationImportance(X2, cv=cv, n_iter=10).fit(X.to_numpy(), Y.to_numpy())

feature_importances_xgb = [(feature, importance) for feature, importance in

                           zip(list(X.columns), list(np.abs(perm_xgb.feature_importances_)))]

feature_importances_xgb = pd.DataFrame(sorted(feature_importances_xgb, key=lambda x: x[1], reverse=True))

feature_importances_xgb.to_csv('feature_importances_xgb_2020_APSIM.csv')

In [95]:
def objective_LGB(params):

    LGB_df_B = pd.DataFrame()

    G1_B = LGBMRegressor(objective='regression', **params)

    for train_index, test_index in kf.split(X):

        LGB_B = G1_B.fit(np.array(X.drop(columns='Year'))[train_index], np.array(Y)[train_index])

        LGB_df_B = pd.concat([LGB_df_B, pd.DataFrame(G1_B.predict(np.array(X.drop(columns='Year'))[test_index]))])

    loss_LGB = mse(data.Yield[(data.Year < 2020)], LGB_df_B)

    return {'loss': loss_LGB, 'params': params, 'status': STATUS_OK}

In [96]:
space_LGB = {'num_leaves': hp.choice('num_leaves', [int(x) for x in np.arange(5, 40, 2)]),

             'learning_rate': hp.uniform('learning_rate', 0.1, 0.5),

             'n_estimators': hp.choice('n_estimators', [500, 1000, 1500, 2000])}

tpe_algorithm = tpe.suggest

trials_LGB = Trials()

best_LGB = fmin(fn=objective_LGB, space=space_LGB, algo=tpe.suggest,

                max_evals=max_evals, trials=trials_LGB, rstate=RandomState(1607))

LGB_param_B = pd.DataFrame({'num_leaves': [], 'learning_rate': [], 'n_estimators': []})

for i in range(max_evals):

    LGB_param_B.num_leaves[i] = trials_LGB.results[i]['params']['num_leaves']

    LGB_param_B.learning_rate[i] = trials_LGB.results[i]['params']['learning_rate']

    LGB_param_B.n_estimators[i] = trials_LGB.results[i]['params']['n_estimators']

LGB_param_B = pd.DataFrame({'num_leaves': LGB_param_B.num_leaves,

                            'learning_rate': LGB_param_B.learning_rate,

                            'n_estimators': LGB_param_B.n_estimators})

100%|███████████████████████████████████████████████| 20/20 [41:14<00:00, 123.72s/trial, best loss: 246.93440119809904]


In [97]:
### ---------- LGB ------------ ###

LGB_df2 = pd.DataFrame()

G2 = LGBMRegressor(objective='regression', random_state=1607,

                   num_leaves=int(trials_LGB.best_trial['result']['params']['num_leaves']),

                   learning_rate=trials_LGB.best_trial['result']['params']['learning_rate'],

                   n_estimators=int(trials_LGB.best_trial['result']['params']['n_estimators']))

for train_index, test_index in kf.split(X):

    G2.fit(np.array(X.drop(columns='Year'))[train_index], np.array(Y)[train_index])

    LGB_df2 = pd.concat([LGB_df2, pd.DataFrame(G2.predict(np.array(X.drop(columns='Year'))[test_index]))])

LGB_df2 = LGB_df2.reset_index(drop=True)

LGB_mse2 = mse(data.Yield[(data.Year<2020)], LGB_df2)
LGB_rmse2 = np.sqrt(LGB_mse2)

LGB = G2.fit(np.array(X.drop(columns=['Year'])),np.array(Y))

LGB_preds_test2 = LGB.predict(x_test.drop(columns=['Year']))

pd.DataFrame(LGB_preds_test2).to_csv('LGB_preds_test_2020_APSIM.csv')

LGB_mse_test2 = mse(data.Yield[data.Year==2020], LGB_preds_test2)

LGB_rmse_test2 = np.sqrt(LGB_mse_test2)

LGB_preds_train = LGB.predict(X.drop(columns=['Year']))

pd.DataFrame(LGB_preds_train).to_csv('LGB_preds_train_2020_APSIM.csv')

LGB_rmse_train = np.sqrt(mse(data.Yield[data.Year<2020], LGB_preds_train))

In [100]:
def objective_RF(params):

    RF_df_B = pd.DataFrame()

    R1_B = RandomForestRegressor(**params)

    for train_index, test_index in kf.split(X):

        RF_B = R1_B.fit(np.array(X.drop(columns='Year'))[train_index], np.array(Y)[train_index])

        RF_df_B = pd.concat([RF_df_B, pd.DataFrame(R1_B.predict(np.array(X.drop(columns='Year'))[test_index]))])

    loss_RF = mse(data.Yield[(data.Year < 2020)], RF_df_B)

    return {'loss': loss_RF, 'params': params, 'status': STATUS_OK}

In [101]:
space_RF = {'n_estimators': hp.choice('n_estimators', [ 300, 400, 500]),

            'max_depth': hp.choice('max_depth', [int(x) for x in np.arange(5, 21, 5)])}

tpe_algorithm = tpe.suggest

trials_RF = Trials()

best_RF = fmin(fn=objective_RF, space=space_RF, algo=tpe.suggest,

               max_evals=max_evals, trials=trials_RF, rstate=RandomState(1607))

RF_param_B = pd.DataFrame({'n_estimators': [], 'max_depth': []})

for i in range(max_evals):

    RF_param_B.n_estimators[i] = trials_RF.results[i]['params']['n_estimators']

    RF_param_B.max_depth[i] = trials_RF.results[i]['params']['max_depth']

RF_param_B = pd.DataFrame({'n_estimators': RF_param_B.n_estimators,

                           'max_depth': RF_param_B.max_depth})

100%|█████████████████████████████████████████████| 20/20 [6:14:16<00:00, 1122.84s/trial, best loss: 290.4914605665918]


In [102]:
### ---------- RF ------------ ###

RF_df2 = pd.DataFrame()

R2 = RandomForestRegressor(max_depth=int(trials_RF.best_trial['result']['params']['max_depth']),

                           n_estimators=int(trials_RF.best_trial['result']['params']['n_estimators']), random_state=1369)

for train_index, test_index in kf.split(X):

    R2.fit(np.array(X.drop(columns='Year'))[train_index], np.array(Y)[train_index])

    RF_df2 = pd.concat([RF_df2, pd.DataFrame(R2.predict(np.array(X.drop(columns='Year'))[test_index]))])

RF_df2 = RF_df2.reset_index(drop=True)

RF_mse2 = mse(data.Yield[(data.Year<2020)], RF_df2)
RF_rmse2 = np.sqrt(RF_mse2)

RF = R2.fit(X.drop(columns='Year'), Y)

RF_preds_test2 = RF.predict(x_test.drop(columns='Year'))

pd.DataFrame(RF_preds_test2).to_csv('RF_preds_test_2020_APSIM.csv')

RF_mse_test2 = mse(data.Yield[data.Year==2020], RF_preds_test2)

RF_rmse_test2 = np.sqrt(RF_mse_test2)

RF_preds_train = RF.predict(X.drop(columns='Year'))

pd.DataFrame(RF_preds_train).to_csv('RF_preds_train_2020_APSIM.csv')

RF_rmse_train = np.sqrt(mse(data.Yield[data.Year<2020], RF_preds_train))


In [86]:
feature_importances_rf = perm_fi(R2, cv, 10)

feature_importances_rf.to_csv('feature_importances_rf_2020_APSIM.csv')

In [517]:
def objective_LASSO(params):

    LASSO_df_B = pd.DataFrame()

    L1_B = Lasso()

    for train_index, test_index in kf.split(X):

        LASSO_B = L1_B.fit(np.array(X.drop(columns='Year'))[train_index], np.array(Y)[train_index])

        LASSO_df_B = pd.concat([LASSO_df_B, pd.DataFrame(LASSO_B.predict(np.array(X.drop(columns='Year'))[test_index]))])

    loss_LASSO = mse(data.Yield[(data.Year < 2020)], LASSO_df_B)

    return {'loss': loss_LASSO, 'params': params, 'status': STATUS_OK}

In [518]:
space_LASSO = {'alpha': hp.uniform('alpha', 10**-5, 1)}

tpe_algorithm = tpe.suggest

trials_LASSO = Trials()

best_LASSO = fmin(fn=objective_LASSO, space=space_LASSO, algo=tpe.suggest,

                  max_evals=max_evals, trials=trials_LASSO, rstate=RandomState(1607))

LASSO_param_B = pd.DataFrame({'alpha': []})

for i in range(max_evals):

    LASSO_param_B.alpha[i] = trials_LASSO.results[i]['params']['alpha']

LASSO_param_B = pd.DataFrame(LASSO_param_B.alpha)

100%|█████████████████████████████████████████████████| 40/40 [04:31<00:00,  6.80s/trial, best loss: 418.1772703194264]


In [519]:
LASSO_df2 = pd.DataFrame()

L2 = Lasso(alpha=trials_LASSO.best_trial['result']['params']['alpha'], random_state=1607)

for train_index, test_index in kf.split(X):

    L2.fit(np.array(X.drop(columns='Year'))[train_index], np.array(Y)[train_index])

    LASSO_df2 = pd.concat([LASSO_df2, pd.DataFrame(L2.predict(np.array(X.drop(columns='Year'))[test_index]))])

LASSO_df2 = LASSO_df2.reset_index(drop=True)

LASSO_mse2 = mse(data.Yield[(data.Year<2020)], LASSO_df2)
LASSO_rmse2 = np.sqrt(LASSO_mse2)

LASSO = L2.fit(X.drop(columns='Year'), Y)

LASSO_preds_test2 = LASSO.predict(x_test.drop(columns='Year'))

pd.DataFrame(LASSO_preds_test2).to_csv('LASSO_preds_test_2020_APSIM.csv')

LASSO_mse_test2 = mse(data.Yield[data.Year==2020], LASSO_preds_test2)

LASSO_rmse_test2 = np.sqrt(LASSO_mse_test2)

LASSO_preds_train = LASSO.predict(X.drop(columns='Year'))

pd.DataFrame(LASSO_preds_train).to_csv('LASSO_preds_train_2020_APSIM.csv')

LASSO_rmse_train = np.sqrt(mse(data.Yield[data.Year<2020], LASSO_preds_train))


In [90]:

feature_importances_lasso = perm_fi(L2, cv, 10)

feature_importances_lasso.to_csv('feature_importances_lasso_2020_APSIM.csv')


In [116]:
LR_df2 = pd.DataFrame()

lm2 = LinearRegression()

lm2.fit(X.drop(columns='Year'),Y)

for train_index, test_index in kf.split(X):

    lm2.fit(np.array(X.drop(columns='Year'))[train_index], np.array(Y)[train_index])

    LR_df2 = pd.concat([LR_df2, pd.DataFrame(lm2.predict(np.array(X.drop(columns='Year'))[test_index]))])

LR_df2 = LR_df2.reset_index(drop=True)

LR_mse2 = mse(data.Yield[(data.Year<2020)], LR_df2)
LR_rmse2 = np.sqrt(LR_mse2)
LR = lm2.fit(X.drop(columns='Year'), Y)

LR_preds_test2 = LR.predict(x_test.drop(columns='Year'))

pd.DataFrame(LR_preds_test2).to_csv('LR_preds_test2_2020_APSIM.csv')

LR_mse_test2 = mse(data.Yield[data.Year==2020], LR_preds_test2)

LR_rmse_test2 = np.sqrt(LR_mse_test2)

LR_preds_train = LR.predict(X.drop(columns='Year'))

pd.DataFrame(LR_preds_train).to_csv('LR_preds_train_2020_APSIM.csv')

LR_rmse_train = np.sqrt(mse(data.Yield[data.Year<2020], LR_preds_train))

feature_importances_lr = perm_fi(lm2, cv, 10)

feature_importances_lr.to_csv('feature_importances_lr_2020_APSIM.csv')

In [122]:
## ---------------- Optimizing Ensembles ---------------- ##

def objective2(y):

    return np.sqrt(mse(data.Yield[(data.Year<2020)],

               (y[0]*LASSO_df2 + y[1]*XGB_df2 + y[2]*LGB_df2 + y[3]*RF_df2 + y[4]*LR_df2)))



def constraint12(y):

    return y[0] + y[1] + y[2] + y[3] + y[4] - 1.0

def constraint22(y):

    return LASSO_rmse2 - objective2(y)

def constraint32(y):

    return XGB_rmse2 - objective2(y)

def constraint42(y):

    return LGB_rmse2 - objective2(y)

def constraint52(y):

    return RF_rmse2 - objective2(y)

def constraint62(y):

    return LR_rmse2 - objective2(y)

In [123]:
y0 = np.zeros(5)

y0[0] = 1 / 5

y0[1] = 1 / 5

y0[2] = 1 / 5

y0[3] = 1 / 5

y0[4] = 1 / 5

b = (0, 1.0)

bnds2 = (b, b, b, b, b)

con12 = {'type': 'eq', 'fun': constraint12}

con22 = {'type': 'ineq', 'fun': constraint22}

con32 = {'type': 'ineq', 'fun': constraint32}

con42 = {'type': 'ineq', 'fun': constraint42}

con52 = {'type': 'ineq', 'fun': constraint52}

con62 = {'type': 'ineq', 'fun': constraint62}

cons2 = [con12, con22, con32, con42, con52, con62]



solution2 = minimize(objective2, y0, method='SLSQP',

                    options={'disp': True, 'maxiter': 5000, 'eps': 1e-5}, bounds=bnds2,

                    constraints=cons2)

y = solution2.x

Optimization terminated successfully    (Exit mode 0)
            Current function value: 246.93440119931574
            Iterations: 2
            Function evaluations: 12
            Gradient evaluations: 2


In [125]:
cowe_preds_test = y[0]*LASSO_preds_test2 + y[1]*XGB_preds_test2 + y[2]*LGB_preds_test2 + y[3]*RF_preds_test2 + y[4]*LR_preds_test2

cowe_mse_test = mse(data.Yield[data.Year==2020], cowe_preds_test)

cowe_rmse_test = np.sqrt(cowe_mse_test)

pd.DataFrame(cowe_preds_test).to_csv('cowe_preds_test_2020_APSIM.csv')

cowe_preds_train = y[0]*LASSO_preds_train + y[1]*XGB_preds_train + y[2]*LGB_preds_train + y[3]*RF_preds_train + y[4]*LR_preds_train

pd.DataFrame(cowe_preds_train).to_csv('cowe_preds_train_2020_APSIM.csv')

cowe_rmse_train = np.sqrt(mse(data.Yield[data.Year<2020], cowe_preds_train))


In [128]:
cowe_preds_CV = y[0]*LASSO_df2 + y[1]*XGB_df2 + y[2]*LGB_df2 + y[3]*RF_df2 + y[4]*LR_df2

cowe_mse_CV = mse(data.Yield[(data.Year<2020)], cowe_preds_CV)

cowe_rmse_CV = np.sqrt(cowe_mse_CV)

In [ ]:
cls_preds_test = y0[0]*LASSO_preds_test2 + y0[1]*XGB_preds_test2 + y0[2]*LGB_preds_test2 + y0[3]*RF_preds_test2 + y0[4]*LR_preds_test2

cls_mse_test = mse(data.Yield[data.Year==2020], cls_preds_test)

cls_rmse_test = np.sqrt(cls_mse_test)

pd.DataFrame(cls_preds_test).to_csv('cls_preds_test_2020_APSIM.csv')

cls_preds_train = y0[0]*LASSO_preds_train + y0[1]*XGB_preds_train + y0[2]*LGB_preds_train + y0[3]*RF_preds_train + y0[4]*LR_preds_train

pd.DataFrame(cls_preds_train).to_csv('cls_preds_train_2020_APSIM.csv')

cls_rmse_train = np.sqrt(mse(data.Yield[data.Year<2020], cls_preds_train))


In [ ]:
cls_preds_CV = y0[0]*LASSO_df2 + y0[1]*XGB_df2 + y0[2]*LGB_df2 + y0[3]*RF_df2 + y0[4]*LR_df2

cls_mse_CV = mse(data.Yield[(data.Year<2020)], cls_preds_CV)

cls_rmse_CV = np.sqrt(cls_mse_CV)

In [ ]:
## -------------------------------- STACKING -------------------------------- ##

predsDF2 = pd.DataFrame()

predsDF2['LASSO'] = LASSO_df2[0]

predsDF2['XGB']= XGB_df2[0]

predsDF2['LGB'] = LGB_df2[0]

predsDF2['RF'] = RF_df2[0]

predsDF2['LR'] = LR_df2[0]

predsDF2['Y'] = data.Yield[(data.Year < 2020)].reset_index(drop=True)

x_stacked2 = predsDF2.drop(columns='Y', axis=1)

y_stacked2 = predsDF2['Y']

testPreds2 = pd.DataFrame([LASSO_preds_test2, XGB_preds_test2, LGB_preds_test2, RF_preds_test2, LR_preds_test2]).T

testPreds2.columns = ['LASSO', 'XGB', 'LGB', 'RF', 'LR']



stck_reg2 = LinearRegression()

stck_reg2.fit(x_stacked2, y_stacked2)

stck_reg_preds_test2 = stck_reg2.predict(testPreds2)

stck_reg_mse_test2 = mse(data.Yield[data.Year == 2020], stck_reg_preds_test2)

stck_reg_rmse_test2 = np.sqrt(stck_reg_mse_test2)

pd.DataFrame(stck_reg_preds_test2).to_csv('stck_reg_preds_test_2020_APSIM.csv')

stck_reg_preds_train = stck_reg2.predict(x_stacked2)

pd.DataFrame(stck_reg_preds_train).to_csv('stck_reg_preds_train_2020_APSIM.csv')

stck_reg_rmse_train = np.sqrt(mse(data.Yield[data.Year < 2020], stck_reg_preds_train))



stck_lasso2 = Lasso()

stck_lasso2.fit(x_stacked2, y_stacked2)

stck_lasso_preds_test2 = stck_lasso2.predict(testPreds2)

stck_lasso_mse_test2 = mse(data.Yield[data.Year == 2020], stck_lasso_preds_test2)

stck_lasso_rmse_test2 = np.sqrt(stck_lasso_mse_test2)

pd.DataFrame(stck_lasso_preds_test2).to_csv('stck_lasso_preds_test_2020_APSIM.csv')

stck_lasso_preds_train = stck_lasso2.predict(x_stacked2)

pd.DataFrame(stck_lasso_preds_train).to_csv('stck_lasso_preds_train_2020_APSIM.csv')

stck_lasso_rmse_train = np.sqrt(mse(data.Yield[data.Year < 2020], stck_lasso_preds_train))



stck_rf2 = RandomForestRegressor()

stck_rf2.fit(x_stacked2, y_stacked2)

stck_rf_preds_test2 = stck_rf2.predict(testPreds2)

stck_rf_mse_test2 = mse(data.Yield[data.Year == 2020], stck_rf_preds_test2)

stck_rf_rmse_test2 = np.sqrt(stck_rf_mse_test2)

pd.DataFrame(stck_rf_preds_test2).to_csv('stck_rf_preds_test_2020_APSIM.csv')

stck_rf_preds_train = stck_rf2.predict(x_stacked2)

pd.DataFrame(stck_rf_preds_train).to_csv('stck_rf_preds_train_2020_APSIM.csv')

stck_rf_rmse_train = np.sqrt(mse(data.Yield[data.Year < 2020], stck_rf_preds_train))



stck_lgb2 = LGBMRegressor()

stck_lgb2.fit(x_stacked2, y_stacked2)

stck_lgb_preds_test2 = stck_lgb2.predict(testPreds2)

stck_lgb_mse_test2 = mse(data.Yield[data.Year == 2020], stck_lgb_preds_test2)

stck_lgb_rmse_test2 = np.sqrt(stck_lgb_mse_test2)

pd.DataFrame(stck_lgb_preds_test2).to_csv('stck_lgb_preds_test_2020_APSIM.csv')

stck_lgb_preds_train = stck_lgb2.predict(x_stacked2)

pd.DataFrame(stck_lgb_preds_train).to_csv('stck_lgb_preds_train_2020_APSIM.csv')

stck_lgb_rmse_train = np.sqrt(mse(data.Yield[data.Year < 2020], stck_lgb_preds_train))


In [ ]:
test_results = pd.DataFrame(data={'model':['RMSE'],'LASSO':[LASSO_rmse_test2], 'XGB':[XGB_rmse_test2], 'LGB':[LGB_rmse_test2],

                                  'RF': [RF_rmse_test2], 'LR': [LR_rmse_test2],

                                  'COWE': [cowe_rmse_test], 'Classical': [cls_rmse_test],

                                  'stck_reg': [stck_reg_rmse_test2], 'stck_lasso': [stck_lasso_rmse_test2],

                                  'stck_rf': [stck_rf_rmse_test2], 'stck_lgb': [stck_lgb_rmse_test2]})



train_results = pd.DataFrame(data={'model':['RMSE'],'LASSO':[LASSO_rmse_train], 'XGB':[XGB_rmse_train], 'LGB':[LGB_rmse_train],

                                  'RF': [RF_rmse_train], 'LR': [LR_rmse_train],

                                  'COWE': [cowe_rmse_train], 'Classical': [cls_rmse_train],

                                  'stck_reg': [stck_reg_rmse_train], 'stck_lasso': [stck_lasso_rmse_train],

                                  'stck_rf': [stck_rf_rmse_train], 'stck_lgb': [stck_lgb_rmse_train]})



CV_results = pd.DataFrame(data={'model':['RMSE'], 'LASSO':[np.sqrt(LASSO_mse2)], 'XGB':[np.sqrt(XGB_mse2)],

                                'LGB':[np.sqrt(LGB_mse2)], 'RF': [np.sqrt(RF_mse2)], 'LR': [np.sqrt(LR_mse2)],

                                'COWE': [cowe_rmse_CV],

                                'Classical':[cls_rmse_CV]})



test_results.to_csv('2020_test_APSIM_Combined.csv')

train_results.to_csv('2020_train_APSIM_Combined.csv')

CV_results.to_csv('2020_CV_APSIM_Combined.csv')

In [131]:
## Combine Results in one DataFrame:
Test_Result_df = pd.DataFrame({'Year':data.Year[(data.Year==2020)],
                       'State':data.State[(data.Year==2020)],
                       'County':data.County[(data.Year==2020)],
                       'Yield':data.Yield[(data.Year==2020)],
                       'RF':RF_preds_test2,
                       'XGBoost':XGB_preds_test2,
                       'LGB':LGB_preds_test2,
                       'Lasso':LASSO_preds_test2,
                       'Linear':LR_preds_test2,
                       'Optimized':cowe_preds_test,
                        'Classical':cls_preds_test})
Test_Result_df.head()

,Year,State,County,Yield,RF,XGBoost,LGB,Lasso,Linear,Optimized
0,2020,17,39,201.6,213.21900,203.146271,197.116499,200.233301,199.468271,197.116499
1,2020,17,107,210.7,211.37275,202.758774,201.161601,196.696661,193.936446,201.161601
2,2020,17,115,210.3,200.31450,197.966995,197.069165,201.049672,201.650277,197.069165
3,2020,17,123,207.1,213.18950,206.360306,206.015466,203.632427,202.917202,206.015466
4,2020,17,125,195.6,185.90900,185.041901,187.876252,185.434772,187.235021,187.876252


In [187]:
Test_State = pd.DataFrame()
Test_State['StateID'] = Test_Result_df.State.unique()
for s in Test_State.StateID.unique():
    Test_State.loc[(Test_State.StateID==s),'Mean_Yield'] = Test_Result_df[Test_Result_df.State==s].Yield.mean()
    Test_State.loc[(Test_State.StateID==s),'RMSE_RF'] = np.sqrt(mse(Test_Result_df[Test_Result_df.State==s].Yield,Test_Result_df[Test_Result_df.State==s].RF))
    Test_State.loc[(Test_State.StateID==s),'RMSE_XGB'] = np.sqrt(mse(Test_Result_df[Test_Result_df.State==s].Yield,Test_Result_df[Test_Result_df.State==s].XGBoost))
    Test_State.loc[(Test_State.StateID==s),'RMSE_LGB'] = np.sqrt(mse(Test_Result_df[Test_Result_df.State==s].Yield,Test_Result_df[Test_Result_df.State==s].LGB))
    Test_State.loc[(Test_State.StateID==s),'RMSE_LASSO'] = np.sqrt(mse(Test_Result_df[Test_Result_df.State==s].Yield,Test_Result_df[Test_Result_df.State==s].Lasso))
    Test_State.loc[(Test_State.StateID==s),'RMSE_LR'] = np.sqrt(mse(Test_Result_df[Test_Result_df.State==s].Yield,Test_Result_df[Test_Result_df.State==s].Linear))
    Test_State.loc[(Test_State.StateID==s),'RMSE_OP_EN'] = np.sqrt(mse(Test_Result_df[Test_Result_df.State==s].Yield,Test_Result_df[Test_Result_df.State==s].Optimized))
    

In [188]:
for s in Test_State.StateID.unique():
    Test_State.loc[(Test_State.StateID==s),'State'] = data[data['State']==s].State.unique()

In [189]:
Test_State.to_csv('Corn_Yield_RMSE_Statewise_2020_APSIM.csv')

,StateID,Mean_Yield,RMSE_RF,RMSE_XGB,RMSE_LGB,RMSE_LASSO,RMSE_LR,RMSE_OP_EN,State
0,17,187.842424,14.210711,11.358839,11.774255,12.131997,12.393215,11.774255,ILLINOIS
1,18,184.638202,15.129554,16.527606,19.662830,21.426475,22.155746,19.662830,INDIANA
2,19,176.027368,26.401677,23.201312,20.264107,20.126691,21.906724,20.264107,IOWA
3,20,134.898889,22.377073,21.920003,22.458970,25.593901,24.837223,22.458970,KANSAS
4,27,179.774026,11.890718,12.159423,12.123883,11.221180,12.883136,12.123883,MINNESOTA
5,29,158.247191,27.227946,23.150972,24.807798,27.169227,27.247745,24.807798,MISSOURI
6,31,182.822973,15.381397,16.314616,18.235163,14.046446,16.144799,18.235163,NEBRASKA
7,39,164.050602,12.292633,14.920687,12.487701,20.313236,18.984219,12.487701,OHIO
8,46,152.737037,15.404426,15.673695,17.679822,15.080204,15.426961,17.679822,SOUTH DAKOTA
9,55,163.850000,10.616341,11.870084,11.525790,11.238485,14.308031,11.525790,WISCONSIN


In [132]:
## Combine Results in one DataFrame:
Train_Result_df = pd.DataFrame({'Year':data.Year[(data.Year<2020)],
                       'State':data.State[(data.Year<2020)],
                       'County':data.County[(data.Year<2020)],
                       'Yield':data.Yield[(data.Year<2020)],
                       'RF':RF_preds_train,
                       'XGBoost':XGB_preds_train,
                       'LGB':LGB_preds_train,
                       'Lasso':LASSO_preds_train,
                       'Linear':LR_preds_train,
                       'Optimized':cowe_preds_train,
                       'Classical':cls_preds_train})

,Year,State,County,Yield,RF,XGBoost,LGB,Lasso,Linear,Optimized
903,2019,17,107,194.2,195.86000,190.931320,192.825342,191.531497,194.087889,192.825342
904,2019,17,115,204.4,201.26750,194.863083,199.478709,193.946424,197.422055,199.478709
905,2019,17,123,184.9,187.78200,188.472031,190.148525,192.388752,193.174909,190.148525
906,2019,17,125,169.0,169.02850,173.132614,175.537868,177.145672,177.506731,175.537868
907,2019,17,113,198.8,196.64125,194.610794,197.336713,193.752568,196.627993,197.336713


In [133]:
Train_Result_df.to_csv('Corn_Train_Result_2020_APSIM.csv')
Test_Result_df.to_csv('Corn_Test_Result_2020_APSIM.csv')

In [147]:
result_combined = pd.DataFrame({'Model':['XGBoost','LGB','RF','Lasso','LR','Opt_Ensemble'],
                               'Test_RMSE':np.round(([XGB_rmse_test2,LGB_rmse_test2,RF_rmse_test2,LASSO_rmse_test2,LR_rmse_test2,cowe_rmse_test]),decimals =2),
                               'Train_RMSE': np.round(([XGB_rmse_train,LGB_rmse_train,RF_rmse_train,LASSO_rmse_train,LR_rmse_train,cowe_rmse_train]),decimals =2),
                               'Test_RRMSE':np.round((([XGB_rmse_test2,LGB_rmse_test2,RF_rmse_test2,LASSO_rmse_test2,LR_rmse_test2,cowe_rmse_test])/data.Yield[data.Year==2020].mean()*100),decimals =2),
                               'Train_RRMSE': np.round((([XGB_rmse_train,LGB_rmse_train,RF_rmse_train,LASSO_rmse_train,LR_rmse_train,cowe_rmse_train])/data.Yield[data.Year<2020].mean()*100),decimals =2)})

In [152]:
result_combined.to_csv('Year_2020_Corp_pred_APSIM_Samira.csv')

,Model,Test_RMSE,Train_RMSE,Test_RRMSE,Train_RRMSE
0,XGBoost,17.46,10.81,10.23,7.90
1,LGB,18.10,7.85,10.61,5.74
2,RF,18.20,3.32,10.66,2.42
3,Lasso,19.36,15.02,11.34,10.98
4,LR,19.90,14.25,11.66,10.42
